<a href="https://colab.research.google.com/github/dddmmm789/youtube_dubbing_tool/blob/main/V2_download_transcribe_translate_from_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube Video Download Service

This notebook:
 - downloads YouTube videos directly to Google Drive
 - extracts audio
 - split audio to vocals and backgroung music
 - transcrive vocals
 - translate vocals
 - createa a vocals json file

 * all the files will be stored in google drive

## Setup
First, we'll install required packages and mount Google Drive:

In [2]:
# @title Mounting Google Drive
# Set up steprs to downloading the video - press play
##video download from youtube - set up 1
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title Play this step and wait to see the message: Successfully installed python-dotenv-1.0.1
#video download from youtube - set up 2
!pip install yt-dlp
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 46.3 MB/s eta 0:00:00


In [ ]:
# @title Import required libraries
# video download from youtube - Import required libraries
import os
import yt_dlp
from IPython.display import HTML, display

# Print success message
print("✅ Successfully imported all required libraries!")
print("Ready to download YouTube videos to Google Drive.")

✅ Successfully imported all required libraries!
Ready to download YouTube videos to Google Drive.


In [ ]:
# @title Download the video from Youtube
import os
import yt_dlp
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.colab import auth
from oauth2client.client import GoogleCredentials

def sanitize_filename(filename):
    """Remove or replace problematic characters in filename."""
    # Replace problematic characters with underscores
    invalid_chars = ['?', '！', '？', '|', ':', '/', '\\', '*', '"', '<', '>', '|']
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    # Replace multiple spaces/underscores with single underscore
    filename = '_'.join(filter(None, filename.split()))
    return filename

def download_video(youtube_url, folder_id):
    """Download a video from YouTube and upload to Google Drive."""
    print(f"🚀 Starting download process...")
    print(f"📍 URL: {youtube_url}")

    # Set up Google Drive service
    credentials = GoogleCredentials.get_application_default()
    drive_service = build('drive', 'v3', credentials=credentials)

    # Verify folder exists
    try:
        folder = drive_service.files().get(fileId=folder_id).execute()
        print(f"📁 Target folder: {folder.get('name', 'Unknown folder')}")
    except Exception as e:
        print(f"❌ Error accessing folder: {str(e)}")
        return False

    # Temporary local directory
    temp_dir = '/content/temp_video'
    os.makedirs(temp_dir, exist_ok=True)

    # Configure yt-dlp options with sanitized output template
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': f'{temp_dir}/%(title)s.%(ext)s',
        'quiet': False,
        'no_warnings': False,
        'progress': True,
        'restrictfilenames': True,  # Restrict filenames to ASCII characters
    }

    try:
        print("🔄 Initializing download...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            # Get video info first
            info = ydl.extract_info(youtube_url, download=False)
            video_title = info['title']
            safe_title = sanitize_filename(video_title)

            # Update output template with safe filename
            ydl_opts['outtmpl'] = f'{temp_dir}/{safe_title}.%(ext)s'

            # Download with updated options
            with yt_dlp.YoutubeDL(ydl_opts) as ydl2:
                ydl2.download([youtube_url])

            # Construct the expected output path
            local_path = os.path.join(temp_dir, f"{safe_title}.mp4")

            print("\n✨ Download Summary:")
            print(f"📝 Original title: {video_title}")
            print(f"📝 Safe title: {safe_title}")
            print(f"📁 Temporary local path: {local_path}")

            # Verify local file exists and get size
            if not os.path.exists(local_path):
                print(f"❌ Error: Downloaded file not found at {local_path}")
                print("📂 Directory contents:")
                for file in os.listdir(temp_dir):
                    print(f"- {file}")
                return False

            file_size = os.path.getsize(local_path) / (1024 * 1024)  # Convert to MB
            print(f"📊 File size: {file_size:.1f} MB")

            # Upload to Google Drive
            print("\n📤 Uploading to Google Drive...")
            file_metadata = {
                'name': f"{video_title}.mp4",  # Use original title for Drive
                'parents': [folder_id],
                'mimeType': 'video/mp4'
            }

            media = MediaFileUpload(
                local_path,
                mimetype='video/mp4',
                resumable=True,
                chunksize=1024*1024
            )

            # Create the file with progress tracking
            request = drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id,name,mimeType,size,parents'
            )

            response = None
            while response is None:
                status, response = request.next_chunk()
                if status:
                    print(f"⏳ Uploaded: {int(status.progress() * 100)}%")

            # Verify upload
            file_id = response.get('id')
            uploaded_file = drive_service.files().get(
                fileId=file_id,
                fields='id,name,size,parents,webViewLink'
            ).execute()

            print("\n✅ Upload verified:")
            print(f"📝 Name: {uploaded_file.get('name')}")
            print(f"📌 ID: {uploaded_file.get('id')}")
            print(f"🔗 Direct link: {uploaded_file.get('webViewLink')}")

            # Clean up temporary file
            os.remove(local_path)
            print("\n🧹 Temporary file cleaned up")

            return uploaded_file.get('id')

    except Exception as e:
        print("\n❌ Upload Failed!")
        print(f"⚠️ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        return False

# Mount Google Drive and authenticate
drive.mount('/content/drive')
auth.authenticate_user()

# Set up the specific Google Drive folder ID
FOLDER_ID = "1etobnRBWhgj3s9HtqUbjUvWizKqs3dFV"

# Get URL and start download
DEFAULT_URL = "https://www.youtube.com/watch?v=AdBzzpq3xV4"
test_url = input("Enter YouTube URL to download (press Enter for default): ").strip()

# Use default URL if no input provided
if not test_url:
    test_url = DEFAULT_URL
    print(f"Using default URL: {DEFAULT_URL}")

file_id = download_video(test_url, FOLDER_ID)
if file_id:
    print(f"\n🎉 Final Result:")
    print(f"✅ Video successfully uploaded to Google Drive")
    print(f"📂 Folder link: https://drive.google.com/drive/folders/{FOLDER_ID}")
    print(f"🎥 Video link: https://drive.google.com/file/d/{file_id}/view")
    print("\n⚠️ Note: It might take a few minutes for the file to appear in the folder view")
else:
    print("\n❌ Upload failed. Please check the error messages above.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter YouTube URL to download (press Enter for default): https://www.youtube.com/shorts/GFV4J50dfRE
🚀 Starting download process...
📍 URL: https://www.youtube.com/shorts/GFV4J50dfRE
📁 Target folder: downloads
🔄 Initializing download...
[youtube] Extracting URL: https://www.youtube.com/shorts/GFV4J50dfRE
[youtube] GFV4J50dfRE: Downloading webpage
[youtube] GFV4J50dfRE: Downloading ios player API JSON
[youtube] GFV4J50dfRE: Downloading mweb player API JSON
[youtube] GFV4J50dfRE: Downloading player b46bb280
[youtube] GFV4J50dfRE: Downloading m3u8 information
[youtube] Extracting URL: https://www.youtube.com/shorts/GFV4J50dfRE
[youtube] GFV4J50dfRE: Downloading webpage
[youtube] GFV4J50dfRE: Downloading ios player API JSON
[youtube] GFV4J50dfRE: Downloading mweb player API JSON
[youtube] GFV4J50dfRE: Downloading m3u8 information
[info] GFV4J50dfRE: Downloading 1 f

In [ ]:
# @title Extract audio main
#extract audio main
import os
import subprocess
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io

# Ensure Google Drive is mounted and authenticated
drive.mount('/content/drive', force_remount=True)
auth.authenticate_user()

# Set up Google Drive service
credentials = GoogleCredentials.get_application_default()
drive_service = build('drive', 'v3', credentials=credentials)

def download_file_from_drive(file_id, destination):
    """Download a file from Google Drive."""
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(destination, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}%.")

def extract_audio(input_file, output_file):
    """Extract audio from video file using ffmpeg."""
    command = [
        'ffmpeg',
        '-i', input_file,
        '-vn',  # Disable video
        '-acodec', 'pcm_s16le',  # Audio codec
        '-ar', '44100',  # Audio sampling rate
        '-ac', '2',  # Number of audio channels
        output_file
    ]
    subprocess.run(command, check=True)

def create_folder_in_drive(folder_name, parent_folder_id):
    """Create a new folder in Google Drive."""
    file_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_folder_id]
    }
    file = drive_service.files().create(body=file_metadata, fields='id').execute()
    return file.get('id')

def upload_to_drive(file_path, folder_id):
    """Upload a file to Google Drive."""
    file_name = os.path.basename(file_path)
    file_metadata = {'name': file_name, 'parents': [folder_id]}
    media = MediaFileUpload(file_path, resumable=True)
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    return file.get('id')

def move_file_in_drive(file_id, new_folder_id):
    """Move a file to a different folder in Google Drive."""
    file = drive_service.files().get(fileId=file_id, fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))
    file = drive_service.files().update(
        fileId=file_id,
        addParents=new_folder_id,
        removeParents=previous_parents,
        fields='id, parents'
    ).execute()

# Set up directories
FOLDER_ID = "1etobnRBWhgj3s9HtqUbjUvWizKqs3dFV"
TEMP_DIR = '/content/temp_audio'
os.makedirs(TEMP_DIR, exist_ok=True)

# List files in the Google Drive folder
results = drive_service.files().list(q=f"'{FOLDER_ID}' in parents", fields="files(id, name)").execute()
files = results.get('files', [])

if not files:
    print('No files found in the specified folder.')
else:
    print("Files in the folder:")
    for file in files:
        print(f"{file['name']} (ID: {file['id']})")

    # Ask user to select a file
    file_id = input("Enter the ID of the video file to process (or press Enter for the last file): ").strip()
    if not file_id and files:
        file_id = files[-1]['id']
        print(f"Using the last file: {files[-1]['name']}")

    if file_id:
        # Get video details
        video_file = drive_service.files().get(fileId=file_id, fields='name').execute()
        video_name = os.path.splitext(video_file['name'])[0]  # Remove extension

        # Create a new folder in Drive
        new_folder_id = create_folder_in_drive(video_name, FOLDER_ID)
        print(f"Created new folder: {video_name}")

        # Download the selected file
        input_file = os.path.join(TEMP_DIR, f'{video_name}.mp4')
        print(f"Downloading video file to: {input_file}")
        download_file_from_drive(file_id, input_file)

        # Extract audio
        output_file = os.path.join(TEMP_DIR, f'{video_name}_audio.wav')
        print(f"Extracting audio to: {output_file}")
        extract_audio(input_file, output_file)

        # Upload extracted audio to the new folder
        print("Uploading extracted audio to Google Drive...")
        audio_file_id = upload_to_drive(output_file, new_folder_id)

        # Move the original video to the new folder
        print("Moving original video to the new folder...")
        move_file_in_drive(file_id, new_folder_id)

        # Clean up
        os.remove(input_file)
        os.remove(output_file)
        print("Temporary files cleaned up.")

        # Provide links and information
        print("\n📁 File Locations:")
        print(f"Folder: https://drive.google.com/drive/folders/{new_folder_id}")
        print(f"Original video: https://drive.google.com/file/d/{file_id}/view")
        print(f"Extracted audio: https://drive.google.com/file/d/{audio_file_id}/view")
        print(f"\n🔗 Main Google Drive folder: https://drive.google.com/drive/folders/{FOLDER_ID}")

    else:
        print("No file selected for processing.")

Mounted at /content/drive
Files in the folder:
Baby Trains For Olympics! 💪🔥 @podgorka_.mp4 (ID: 1xtcKw3p8QxuhoJjGO2nYBMUmbVy-97M9)
Kunjungilah Ibumu, Dia Pasti Sangat Bahagia 😊 (ID: 1fzYlxpEDCeXQk3S2eavJsITLa-fsRiEO)
Who is the Quietest Loud?? 🤫 | The Loud House (ID: 1_I1-jxi2kxQCYaC50_vxDbAdZsk_4BpY)
Lady Gaga, Bruno Mars - Die With A Smile (ID: 1dtIDjkvh48mWSMUTLBImagDjncrBUvQ3)
גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית! (ID: 1hykZ2dvme9bQQCdc4l32BMB-UWyWB2VH)
extracted_audio_stems (ID: 1zdpu0QpIC0DvD5rdqhde4bfZf7ZPlwKD)
Lady Gaga, Bruno Mars - Die With A Smile (ID: 1mFUO_aqE8on0wvcjA0LCorcmCn0qyMQd)
extracted_audio.wav (ID: 1LugVyKDVKmDFnyRYjs-QGOMrVt2sN5TX)
extracted_audio.wav (ID: 1iIDgvvmCrLM08rYZgI51baiUG9phRION)
The Internet is starting to Break - Here's Why..mp4 (ID: 1tFQsP55VrZRz0r_AcQ6pbd5xrs4_swir)
Lady Gaga, Bruno Mars - Die With A Smile.mp4 (ID: 1bzw8zix-8MfyDebNQCYEA_bPw-hUfThB)
Enter the ID of the video file to process (or press Enter for the last file): 1xtcKw3p8Q

In [ ]:
# @title Split file to audio and vocals (setting up libraries - SPLEETER)
#spleeter set up

from google.colab import drive, auth
import os
import time
import json
from datetime import datetime
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from google.auth import default
import io
import shutil
import subprocess

# Mount Drive and install Spleeter
drive.mount('/content/drive', force_remount=True)
!pip install spleeter

class AudioSplitter:
    def __init__(self):
        # Authenticate
        auth.authenticate_user()

        # Get credentials
        creds, _ = default()

        # Build the Drive service
        self.drive_service = build('drive', 'v3', credentials=creds)

        # Set the specific folder ID and create temp directories
        self.TARGET_FOLDER_ID = "1etobnRBWhgj3s9HtqUbjUvWizKqs3dFV"
        self.TEMP_DIR = '/content/temp_processing'
        os.makedirs(self.TEMP_DIR, exist_ok=True)

    def list_files_recursive(self, folder_id=None):
        """List all WAV files in the folder and its subfolders"""
        if folder_id is None:
            folder_id = self.TARGET_FOLDER_ID

        try:
            # Query for all WAV files in this folder and subfolders
            query = f"'{folder_id}' in parents and (mimeType contains 'audio/wav' or mimeType contains 'audio/x-wav')"
            results = self.drive_service.files().list(
                q=query,
                fields="files(id, name, modifiedTime, parents)",
                orderBy='modifiedTime desc'
            ).execute()

            files = results.get('files', [])

            # Get subfolders
            folder_query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.folder'"
            folder_results = self.drive_service.files().list(
                q=folder_query,
                fields="files(id)"
            ).execute()

            # Recursively search subfolders
            for folder in folder_results.get('files', []):
                files.extend(self.list_files_recursive(folder['id']))

            return files

        except Exception as e:
            print(f"Error listing files: {str(e)}")
            return []

    def process_audio(self, file_id, file_name, parent_folder_id):
        """Process audio file with Spleeter"""
        try:
            input_path = os.path.join(self.TEMP_DIR, file_name)
            output_dir = os.path.join(self.TEMP_DIR, 'output')
            os.makedirs(output_dir, exist_ok=True)

            # Download file
            print(f"📥 Downloading {file_name}...")
            request = self.drive_service.files().get_media(fileId=file_id)
            with open(input_path, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%")

            # Process with Spleeter
            print("🔄 Processing with Spleeter...")
            subprocess.run([
                'spleeter', 'separate',
                '-p', 'spleeter:2stems',
                '-o', output_dir,
                input_path
            ], check=True)

            # Upload results
            base_name = os.path.splitext(file_name)[0]
            stems_path = os.path.join(output_dir, base_name)

            # Create stems folder in the same location as the original file
            stems_folder_name = f"{base_name}_stems"
            folder_metadata = {
                'name': stems_folder_name,
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [parent_folder_id]
            }
            stems_folder = self.drive_service.files().create(
                body=folder_metadata,
                fields='id'
            ).execute()

            # Upload each stem
            for stem in ['vocals', 'accompaniment']:
                stem_file = os.path.join(stems_path, f"{stem}.wav")
                if os.path.exists(stem_file):
                    print(f"📤 Uploading {stem}.wav...")
                    file_metadata = {
                        'name': f"{stem}.wav",
                        'parents': [stems_folder.get('id')]
                    }
                    media = MediaFileUpload(stem_file, resumable=True)
                    self.drive_service.files().create(
                        body=file_metadata,
                        media_body=media,
                        fields='id'
                    ).execute()

            # Cleanup
            shutil.rmtree(self.TEMP_DIR)
            os.makedirs(self.TEMP_DIR, exist_ok=True)

            print("✨ Processing complete!")
            return stems_folder.get('id')

        except Exception as e:
            print(f"❌ Error: {str(e)}")
            return None

Mounted at /content/drive


In [ ]:
# @title Spleeter Main
# Initialize the splitter
splitter = AudioSplitter()

# List all WAV files in the folder and subfolders
files = splitter.list_files_recursive()

if files:
    print("\n🎵 Available Audio Files:")
    for idx, file in enumerate(files, 1):
        modified_time = datetime.fromisoformat(file['modifiedTime'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{idx}. {file['name']} (Modified: {modified_time})")

    while True:
        choice = input("\nEnter the number of the file to process (or 'q' to quit): ").strip()
        if choice.lower() == 'q':
            break

        try:
            idx = int(choice) - 1
            if 0 <= idx < len(files):
                selected_file = files[idx]
                print(f"\n🎯 Processing: {selected_file['name']}")

                stems_folder_id = splitter.process_audio(
                    selected_file['id'],
                    selected_file['name'],
                    selected_file['parents'][0]
                )

                if stems_folder_id:
                    print("\n✅ Success! You can find the stems here:")
                    print(f"🔗 https://drive.google.com/drive/folders/{stems_folder_id}")
                break
            else:
                print("❌ Invalid number. Please try again.")
        except ValueError:
            print("❌ Please enter a valid number.")
else:
    print("No WAV files found in the folder or its subfolders.")


🎵 Available Audio Files:
1. extracted_audio.wav (Modified: 2024-11-27 14:46:30)
2. extracted_audio.wav (Modified: 2024-11-27 14:43:42)
3. Baby Trains For Olympics! 💪🔥 @podgorka__audio.wav (Modified: 2024-12-02 12:12:56)
4. Kunjungilah Ibumu, Dia Pasti Sangat Bahagia 😊_audio.wav (Modified: 2024-12-02 04:58:39)
5. accompaniment.wav (Modified: 2024-12-02 05:01:41)
6. vocals.wav (Modified: 2024-12-02 05:01:40)
7. Who is the Quietest Loud?? 🤫 | The Loud House_audio.wav (Modified: 2024-11-28 06:09:59)
8. accompaniment.wav (Modified: 2024-11-28 06:15:33)
9. vocals.wav (Modified: 2024-11-28 06:15:31)
10. Lady Gaga, Bruno Mars - Die With A Smile_audio.wav (Modified: 2024-11-28 05:52:54)
11. גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית!_audio.wav (Modified: 2024-11-27 16:00:42)
12. accompaniment.wav (Modified: 2024-11-27 16:01:54)
13. vocals.wav (Modified: 2024-11-27 16:01:52)
14. accompaniment.wav (Modified: 2024-11-27 15:24:59)
15. vocals.wav (Modified: 2024-11-27 15:24:57)
16. Lady Gaga, B

In [ ]:
# @title Transcribing the original vocals - OpenAI Whispper set up
#transcription set up

from google.colab import drive, auth
import os
import json
from datetime import datetime
import pandas as pd
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth import default
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io

# Mount Google Drive if not already mounted
drive.mount('/content/drive', force_remount=True)

# Install whisper
!pip install -U openai-whisper

# Now import whisper after installation
import whisper

class TranscriptionService:
    def __init__(self):
        print("Initializing TranscriptionService...")  # Debug print

        # Authenticate and set up Google Drive
        auth.authenticate_user()
        creds, _ = default()
        self.drive_service = build('drive', 'v3', credentials=creds)

        # Target folder ID
        self.TARGET_FOLDER_ID = "1etobnRBWhgj3s9HtqUbjUvWizKqs3dFV"

        # Initialize Whisper model
        print("Loading Whisper model...")
        self.model = whisper.load_model("base")
        print("✅ Model loaded!")

        # Set up working directory
        self.temp_dir = '/content/temp_transcription'
        os.makedirs(self.temp_dir, exist_ok=True)

        # Add language options
        print("Setting up language options...")  # Debug print
        self.LANGUAGE_OPTIONS = {
            "auto": "Auto-detect",
            "en": "English",
            "es": "Spanish",
            "fr": "French",
            "de": "German",
            "it": "Italian",
            "pt": "Portuguese",
            "nl": "Dutch",
            "pl": "Polish",
            "ru": "Russian",
            "ar": "Arabic",
            "zh": "Chinese",
            "ja": "Japanese",
            "ko": "Korean",
            "hi": "Hindi",
            "ID": "Indonesian",
            "he": "Hebrew"
        }
        print("Language options set up successfully")  # Debug print

    def get_vocal_files(self):
        """Get all vocals.wav files from stems folders"""
        try:
            # First get all stems folders
            stems_query = f"mimeType='application/vnd.google-apps.folder' and name contains '_stems'"
            folders = self.drive_service.files().list(
                q=stems_query,
                orderBy='modifiedTime desc',
                fields="files(id, name, modifiedTime)"
            ).execute().get('files', [])

            vocal_files = []
            for folder in folders:
                # Look for vocals.wav in each stems folder
                vocals_query = f"'{folder['id']}' in parents and name='vocals.wav'"
                vocals = self.drive_service.files().list(
                    q=vocals_query,
                    fields="files(id, name)"
                ).execute().get('files', [])

                if vocals:
                    vocal_file = vocals[0]
                    vocal_file['folder_name'] = folder['name']
                    vocal_file['modifiedTime'] = folder['modifiedTime']
                    vocal_files.append(vocal_file)

            return vocal_files
        except Exception as e:
            print(f"Error listing files: {str(e)}")
            return []

    def transcribe_audio(self, file_id, file_name, folder_name):
        """Download and transcribe the audio file"""
        try:
            print("Starting transcription process...")  # Debug print
            print(f"Language options available: {self.LANGUAGE_OPTIONS}")  # Debug print

            # Download file
            input_path = os.path.join(self.temp_dir, file_name)
            request = self.drive_service.files().get_media(fileId=file_id)

            print(f"📥 Downloading {folder_name} - vocals...")
            with open(input_path, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%")

            # Ask for language
            print("\n🌍 Available languages:")
            for code, name in self.LANGUAGE_OPTIONS.items():
                print(f"{code}: {name}")

            language = input("\nEnter language code (or press Enter for auto-detection): ").strip().lower()

            # Validate language input
            if language and language not in self.LANGUAGE_OPTIONS:
                print("❌ Invalid language code, falling back to auto-detection")
                language = None

            # Transcribe
            print("🎯 Transcribing audio...")
            transcription_options = {}
            if language and language != "auto":
                transcription_options["language"] = language
                print(f"Using specified language: {self.LANGUAGE_OPTIONS[language]}")
            else:
                print("Using language auto-detection...")

            result = self.model.transcribe(input_path, **transcription_options)

            # Add detected language to output
            detected_language = result.get('language', 'unknown')
            print(f"🔍 Detected language: {self.LANGUAGE_OPTIONS.get(detected_language, detected_language)}")

            # Create transcription file with additional metadata
            transcript = {
                'text': result['text'],
                'segments': result['segments'],
                'language': detected_language,
                'specified_language': language if language and language != "auto" else None,
                'timestamp': datetime.now().isoformat()
            }

            # Save locally first
            json_path = os.path.join(self.temp_dir, 'transcription.json')
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(transcript, f, ensure_ascii=False, indent=2)

            # Upload to Drive in same folder as vocals
            parent_folder_id = self.drive_service.files().get(
                fileId=file_id,
                fields='parents'
            ).execute()['parents'][0]

            file_metadata = {
                'name': 'transcription.json',
                'parents': [parent_folder_id]
            }

            media = MediaFileUpload(
                json_path,
                mimetype='application/json',
                resumable=True
            )

            print("📤 Uploading transcription...")
            uploaded_file = self.drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()

            # Cleanup
            os.remove(input_path)
            os.remove(json_path)

            return uploaded_file.get('id'), transcript

        except Exception as e:
            print(f"❌ Error during transcription: {str(e)}")
            return None, None

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 11.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=9e3cc646ce8e329e1cfc4a0cb189e950b777fe75fde5c826f509c62dec204cdc
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [ ]:
# @title Transpcription Main
#main transcription

# Initialize the service
transcriber = TranscriptionService()

# Get list of vocal files
vocal_files = transcriber.get_vocal_files()

if vocal_files:
    print("\n🎵 Available Vocal Files:")
    for idx, file in enumerate(vocal_files, 1):
        modified_time = datetime.fromisoformat(file['modifiedTime'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{idx}. {file['folder_name']} (Modified: {modified_time})")

    while True:
        choice = input("\nEnter the number of the file to process (or 'q' to quit): ").strip()
        if choice.lower() == 'q':
            break

        try:
            idx = int(choice) - 1
            if 0 <= idx < len(vocal_files):
                selected_file = vocal_files[idx]
                print(f"\n🎯 Processing: {selected_file['folder_name']}")

                file_id, transcript = transcriber.transcribe_audio(
                    selected_file['id'],
                    selected_file['name'],
                    selected_file['folder_name']
                )

                if file_id:
                    print("\n✅ Success! Transcription complete!")
                    print(f"🔗 Transcription file: https://drive.google.com/file/d/{file_id}/view")
                    print("\n📝 First few lines of transcription:")
                    print(transcript['text'][:200] + "...")
                break
            else:
                print("❌ Invalid number. Please try again.")
        except ValueError:
            print("❌ Please enter a valid number.")
else:
    print("No vocal files found in stems folders.")

Initializing TranscriptionService...
Loading Whisper model...


100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 104MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

✅ Model loaded!
Setting up language options...
Language options set up successfully

🎵 Available Vocal Files:
1. Baby Trains For Olympics! 💪🔥 @podgorka__audio_stems (Modified: 2024-12-02 12:17:09)
2. Kunjungilah Ibumu, Dia Pasti Sangat Bahagia 😊_audio_stems (Modified: 2024-12-02 05:01:38)
3. Who is the Quietest Loud?? 🤫 | The Loud House_audio_stems (Modified: 2024-11-28 06:15:29)
4. גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית!_audio_stems (Modified: 2024-11-27 16:01:51)
5. Lady Gaga, Bruno Mars - Die With A Smile_audio_stems (Modified: 2024-11-27 15:28:23)
6. extracted_audio_stems (Modified: 2024-11-27 15:24:55)

Enter the number of the file to process (or 'q' to quit): 1

🎯 Processing: Baby Trains For Olympics! 💪🔥 @podgorka__audio_stems
Starting transcription process...
Language options available: {'auto': 'Auto-detect', 'en': 'English', 'es': 'Spanish', 'fr': 'French', 'de': 'German', 'it': 'Italian', 'pt': 'Portuguese', 'nl': 'Dutch', 'pl': 'Polish', 'ru': 'Russian', 'ar': 'Arabi

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


🔍 Detected language: English
📤 Uploading transcription...

✅ Success! Transcription complete!
🔗 Transcription file: https://drive.google.com/file/d/1gb6qG9aaYnRb2Cqphi4BJbA_8duoJhkJ/view

📝 First few lines of transcription:
 This baby was already training for the Olympics at 6 months old. The older he got, the more skills he developed as he got stronger and stronger. At just 21 months old, he was already able to flip his...


**Translation**

In [ ]:
# @title Translation with deepL and Google Translate as backup - set up
from google.colab import drive, auth
import os
import json
from datetime import datetime
from googleapiclient.discovery import build
from google.auth import default
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import io

# Mount Google Drive if not already mounted
drive.mount('/content/drive', force_remount=True)

# Install required packages
!pip install deepl
!pip install googletrans==4.0.0-rc1

# Import required libraries
import deepl
from googletrans import Translator

class TranslationService:
    def __init__(self, auth_key):
        try:
            # Initialize DeepL
            self.deepl_translator = deepl.Translator(auth_key)
            print("DeepL API connected successfully!")
            print(f"Character usage: {self.deepl_translator.get_usage().character.count}/{self.deepl_translator.get_usage().character.limit}")

            # Initialize Google Translator as backup
            self.google_translator = Translator()

            # Authenticate and set up Google Drive
            auth.authenticate_user()
            creds, _ = default()
            self.drive_service = build('drive', 'v3', credentials=creds)

            # Set up working directory
            self.temp_dir = '/content/temp_translation'
            os.makedirs(self.temp_dir, exist_ok=True)

            # Define supported languages
            self.DEEPL_LANGUAGES = {
                "EN-US": "English (American)",
                "EN-GB": "English (British)",
                "ES": "Spanish",
                "FR": "French",
                "DE": "German",
                "IT": "Italian",
                "PT-PT": "Portuguese",
                "RU": "Russian",
                "JA": "Japanese",
                "ZH": "Chinese"
            }

            # Additional languages supported by Google Translate
            self.GOOGLE_LANGUAGES = {
                "HE": "Hebrew",
                # Add other languages as needed
            }

            # Combine all supported languages
            self.LANGUAGE_OPTIONS = {**self.DEEPL_LANGUAGES, **self.GOOGLE_LANGUAGES}

        except Exception as e:
            print(f"Error initializing translation services: {str(e)}")
            raise

    def translate_text(self, text, target_lang, source_lang=None):
        """Translate text using appropriate service"""
        try:
            # Handle deprecated EN code
            if target_lang == "EN":
                target_lang = "EN-US"  # Default to US English
                print("Note: Using EN-US for English translation")

            # Use DeepL for supported languages (except when source is Hebrew)
            if target_lang in self.DEEPL_LANGUAGES and source_lang != "HE":
                result = self.deepl_translator.translate_text(text, target_lang=target_lang)
                return result.text
            # Fall back to Google Translate for Hebrew or other unsupported languages
            else:
                # Convert language codes for Google Translate
                google_target_lang = target_lang.lower().split('-')[0]  # Convert EN-US to en, etc.
                google_source_lang = source_lang.lower() if source_lang else 'auto'

                print(f"Using Google Translate: {google_source_lang} -> {google_target_lang}")
                result = self.google_translator.translate(
                    text=text,
                    dest=google_target_lang,
                    src=google_source_lang
                )
                return result.text
        except Exception as e:
            print(f"Translation error: {str(e)}")
            raise

    def list_transcription_files(self):
        """List all transcription.json files in the target folder"""
        try:
            query = f"mimeType='application/json' and name contains 'transcription'"
            results = self.drive_service.files().list(
                q=query,
                orderBy='modifiedTime desc',
                fields="files(id, name, modifiedTime, parents)"
            ).execute()

            files = results.get('files', [])
            if not files:
                print("No transcription files found.")
                return []

            # Filter files that are in our target folder structure
            valid_files = []
            for file in files:
                try:
                    parent = self.drive_service.files().get(
                        fileId=file['parents'][0],
                        fields="name, parents"
                    ).execute()
                    if '_stems' in parent['name']:
                        file['folder_name'] = parent['name']
                        valid_files.append(file)
                except Exception:
                    continue

            return valid_files
        except Exception as e:
            print(f"Error listing files: {str(e)}")
            return []

    def translate_transcription(self, file_id, file_name, folder_name, target_lang, source_lang=None):
        """Download, translate, and upload the transcription file"""
        try:
            # Download file
            input_path = os.path.join(self.temp_dir, file_name)
            request = self.drive_service.files().get_media(fileId=file_id)

            print(f"📥 Downloading {folder_name} - transcription...")
            with open(input_path, 'wb') as f:
                downloader = MediaIoBaseDownload(f, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%")

            # Load transcription
            with open(input_path, 'r', encoding='utf-8') as f:
                transcript = json.load(f)

            # Determine which translation service to use
            service_name = "DeepL" if target_lang in self.DEEPL_LANGUAGES and source_lang != "HE" else "Google Translate"
            print(f"🌐 Using {service_name} for translation to {self.LANGUAGE_OPTIONS[target_lang]}...")

            # Translate each segment
            translated_segments = []
            total_segments = len(transcript['segments'])

            for idx, segment in enumerate(transcript['segments'], 1):
                print(f"Translating segment {idx}/{total_segments}...")
                translated_text = self.translate_text(
                    segment['text'],
                    target_lang,
                    source_lang
                )
                translated_segments.append({
                    "id": segment["id"],
                    "source_text": segment["text"],
                    "translated_text": translated_text,
                    "start": segment["start"],
                    "end": segment["end"],
                    "words": segment.get("words", [])  # Use an empty list if "words" is missing
                })

            # Create translated transcription file
            translated_transcript = {
                "metadata": {
                    "source_language": source_lang or transcript.get("language", "unknown"),
                    "target_language": target_lang,
                    "translation_service": service_name,
                    "timestamp": datetime.now().isoformat()
                },
                "segments": translated_segments
            }

            # Save locally first
            translated_file_name = f"transcription_{target_lang.lower()}.json"
            json_path = os.path.join(self.temp_dir, translated_file_name)
            with open(json_path, 'w', encoding='utf-8') as f:
                json.dump(translated_transcript, f, ensure_ascii=False, indent=2)

            # Upload to Drive in same folder as original transcription
            parent_folder_id = self.drive_service.files().get(
                fileId=file_id,
                fields='parents'
            ).execute()['parents'][0]

            file_metadata = {
                'name': translated_file_name,
                'parents': [parent_folder_id]
            }

            media = MediaFileUpload(
                json_path,
                mimetype='application/json',
                resumable=True
            )

            print("📤 Uploading translated transcription...")
            uploaded_file = self.drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()

            # Get folder link
            folder_link = f"https://drive.google.com/drive/folders/{parent_folder_id}"

            # Cleanup
            os.remove(input_path)
            os.remove(json_path)

            return uploaded_file.get('id'), translated_transcript, folder_link

        except Exception as e:
            print(f"❌ Error during translation: {str(e)}")
            return None, None, None

Mounted at /content/drive


In [ ]:
# @title Translation Run - json output
# Initialize the service with DeepL API key
auth_key = "bd006995-e700-4b10-8534-92668f452fbf:fx"
translator = TranslationService(auth_key)

# Get list of transcription files
transcription_files = translator.list_transcription_files()

if transcription_files:
    print("\n📄 Available Transcription Files:")
    for idx, file in enumerate(transcription_files, 1):
        modified_time = datetime.fromisoformat(file['modifiedTime'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{idx}. {file['folder_name']} (Modified: {modified_time})")

    # First, select the file
    while True:
        choice = input("\nEnter the number of the file to translate (or 'q' to quit): ").strip()
        if choice.lower() == 'q':
            break

        try:
            idx = int(choice) - 1
            if 0 <= idx < len(transcription_files):
                selected_file = transcription_files[idx]
                print(f"\n🎯 Selected: {selected_file['folder_name']}")

                # Show available languages
                print("\n🌍 Available languages:")
                print("\nDeepL supported languages:")
                print("EN-US: English (American)")
                print("EN-GB: English (British)")
                for code, name in translator.DEEPL_LANGUAGES.items():
                    if not code.startswith("EN"):  # Skip English as we've already shown it
                        print(f"{code}: {name}")
                print("\nAdditional languages (via Google Translate):")
                for code, name in translator.GOOGLE_LANGUAGES.items():
                    print(f"{code}: {name}")

                # Language selection loop
                while True:
                    # Get source and target languages
                    source_lang = input("\nEnter source language code (or press Enter for auto-detection): ").strip().upper()
                    if source_lang == "":
                        source_lang = None
                    elif source_lang not in translator.LANGUAGE_OPTIONS:
                        print("❌ Invalid source language code. Please try again.")
                        continue

                    target_lang = input("Enter target language code (for English, use EN-US or EN-GB): ").strip().upper()
                    if target_lang not in translator.LANGUAGE_OPTIONS:
                        print("❌ Invalid target language code. Please try again.")
                        continue

                    print(f"\nTranslating from {source_lang if source_lang else 'auto-detected'} to {target_lang}...")

                    file_id, translated_transcript, folder_link = translator.translate_transcription(
                        selected_file['id'],
                        selected_file['name'],
                        selected_file['folder_name'],
                        target_lang,
                        source_lang
                    )

                    if file_id:
                        print("\n✅ Success! Translation complete!")
                        print(f"🔗 Translated file: https://drive.google.com/file/d/{file_id}/view")
                        print(f"📂 Folder link: {folder_link}")
                        print("\n📝 First segment translation:")
                        first_segment = translated_transcript['segments'][0]
                        print(f"Original: {first_segment['source_text']}")
                        print(f"Translated: {first_segment['translated_text']}")
                        break  # Break the language selection loop
                break  # Break the file selection loop
            else:
                print("❌ Invalid number. Please try again.")
        except ValueError:
            print("❌ Please enter a valid number.")
else:
    print("No transcription files found.")

DeepL API connected successfully!
Character usage: 7618/500000

📄 Available Transcription Files:
1. Baby Trains For Olympics! 💪🔥 @podgorka__audio_stems (Modified: 2024-12-02 12:21:08)
2. Kunjungilah Ibumu, Dia Pasti Sangat Bahagia 😊_audio_stems (Modified: 2024-12-02 05:11:29)
3. Kunjungilah Ibumu, Dia Pasti Sangat Bahagia 😊_audio_stems (Modified: 2024-12-02 05:03:18)
4. Who is the Quietest Loud?? 🤫 | The Loud House_audio_stems (Modified: 2024-11-28 06:24:15)
5. Who is the Quietest Loud?? 🤫 | The Loud House_audio_stems (Modified: 2024-11-28 06:21:51)
6. גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית!_audio_stems (Modified: 2024-11-27 16:54:01)
7. גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית!_audio_stems (Modified: 2024-11-27 16:52:30)
8. גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית!_audio_stems (Modified: 2024-11-27 16:44:26)
9. גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית!_audio_stems (Modified: 2024-11-27 16:34:39)
10. גמר הישרדות אולסטארס - סיפור הגרר של דן ומירית!_audio_stems 